In [17]:
PET = "../4P/input/core_data_files/projected_by_linear_model_to_2050/petprojected_amounts_to_relog_grouped_2050.csv"
PAPER = "../4P/input/core_data_files/projected_by_linear_model_to_2050/paperprojected_amounts_to_relog_grouped_2050.csv"
OTHER = "../4P/input/core_data_files/projected_by_linear_model_to_2050/otherprojected_amounts_to_relog_grouped_2050.csv"
IRON = "../4P/input/core_data_files/projected_by_linear_model_to_2050/ironprojected_amounts_to_relog_grouped_2050.csv"
HDPE = "../4P/input/core_data_files/projected_by_linear_model_to_2050/hdpeprojected_amounts_to_relog_grouped_2050.csv"
GLASS = "../4P/input/core_data_files/projected_by_linear_model_to_2050/glassprojected_amounts_to_relog_grouped_2050.csv"
FILM = "../4P/input/core_data_files/projected_by_linear_model_to_2050/filmprojected_amounts_to_relog_grouped_2050.csv"
CARD = "../4P/input/core_data_files/projected_by_linear_model_to_2050/cardboardprojected_amounts_to_relog_grouped_2050.csv"
ALM = "../4P/input/core_data_files/projected_by_linear_model_to_2050/aluminumprojected_amounts_to_relog_grouped_2050.csv"
ALL_WASTE_FILES = [PET, PAPER, OTHER, IRON, HDPE, GLASS, FILM, CARD, ALM]

In [18]:
import pandas as pd

In [19]:
pd.read_csv(PET).head()

,Unnamed: 0,name,State,State_County,latitude (deg),longitude (deg),2004.0,2005.0,2006.0,2007.0,...,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050
0,0,Abbeville,South Carolina,South Carolina_Abbeville,34.213809,-82.460460,177.339712,179.135457,180.922328,182.695340,...,235.629661,237.173252,238.716842,240.260432,241.804022,243.347613,244.891203,246.434793,247.978384,249.521974
1,1,Acadia,Louisiana,Louisiana_Acadia,30.291497,-92.411037,485.163465,487.063463,488.914049,490.686005,...,530.055087,531.189758,532.324428,533.459099,534.593770,535.728441,536.863111,537.997782,539.132453,540.267124
2,2,Accomack,Virginia,Virginia_Accomack,37.765944,-75.757807,272.786855,276.143577,279.504906,282.855960,...,395.536995,398.849274,402.161553,405.473833,408.786112,412.098391,415.410671,418.722950,422.035229,425.347509
3,3,Ada,Idaho,Idaho_Ada,43.447861,-116.244456,2564.278612,2603.503427,2643.481321,2683.886637,...,4093.481789,4134.929966,4176.378142,4217.826318,4259.274495,4300.722671,4342.170847,4383.619024,4425.067200,4466.515376
4,4,Adair,Iowa,Iowa_Adair,41.328528,-94.478164,58.150904,58.327650,58.496943,58.653859,...,58.616423,58.605877,58.595331,58.584786,58.574240,58.563694,58.553148,58.542602,58.532056,58.521510


In [20]:
import pandas as pd
import numpy as np

def combine_waste_data(file_paths):
    all_data = []
    waste_types = ['PET', 'Paper', 'Other', 'Iron', 'HDPE', 'Glass', 'Film', 'CardBoard', 'Aluminum']
    
    for file_path, waste_type in zip(file_paths, waste_types):
        df = pd.read_csv(file_path)
        
        if 'Unnamed: 0' in df.columns:
            df = df.drop('Unnamed: 0', axis=1)
        
        year_columns = df.columns[df.columns.str.isnumeric()]
        
        df_melted = df.melt(id_vars=['name', 'State', 'State_County', 'latitude (deg)', 'longitude (deg)'],
                            value_vars=year_columns,
                            var_name='Year', value_name=waste_type)
        
        df_melted['Year'] = df_melted['Year'].astype(int)
        
        all_data.append(df_melted)
    
    combined_df = all_data[0]
    for df in all_data[1:]:
        combined_df = pd.merge(combined_df, df, on=['name', 'State', 'State_County', 'latitude (deg)', 'longitude (deg)', 'Year'])
    
    combined_df = combined_df.rename(columns={'name': 'County', 'State_County': 'State_County'})
    columns_order = ['County', 'State', 'State_County', 'latitude (deg)', 'longitude (deg)', 'Year'] + waste_types
    combined_df = combined_df[columns_order]
    
    return combined_df

ALL_WASTE_FILES = [PET, PAPER, OTHER, IRON, HDPE, GLASS, FILM, CARD, ALM]
combined_waste_df = combine_waste_data(ALL_WASTE_FILES)

print(combined_waste_df.head())


      County           State              State_County  latitude (deg)  \
0  Abbeville  South Carolina  South Carolina_Abbeville       34.213809   
1     Acadia       Louisiana          Louisiana_Acadia       30.291497   
2   Accomack        Virginia         Virginia_Accomack       37.765944   
3        Ada           Idaho                 Idaho_Ada       43.447861   
4      Adair            Iowa                Iowa_Adair       41.328528   

   longitude (deg)  Year          PET        Paper       Other        Iron  \
0       -82.460460  2031   220.193758   127.173331    8.007046   14.012330   
1       -92.411037  2031   518.708380   721.519789   28.817132   11.526853   
2       -75.757807  2031   362.414202   319.896936    9.884024   65.893491   
3      -116.244456  2031  3679.000026  8275.894728  122.633334  327.022225   
4       -94.478164  2031    58.721882   431.013817    2.135341    4.804518   

          HDPE        Glass        Film     CardBoard    Aluminum  
0   120.105686    

In [21]:
columns = ["County", "State", "Year", "PET", "Paper", "Other", "Iron", "HDPE", "Glass", "Film", "Card", "ALM"]

In [16]:
combined_waste_df.to_csv('tt')